# Image Creation with Deep Convolutional GANs

#### Part 1: Data Preprocessing

In [1]:
# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
# Setting some hyperparameters
batchSize = 64 
imageSize = 64 

In [3]:
# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

C:\Users\amira\Anaconda3\envs\face\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [4]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2)

Files already downloaded and verified


In [5]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

#### Part 2: Building the Model

In [6]:
# Defining the generator

class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

# Creating the generator
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [7]:
# Defining the discriminator

class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

# Creating the discriminator
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU(0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU(0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU(0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

#### Part 3: Training the Model

In [ ]:
# Training the DCGANs

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

for epoch in range(1):

    for i, data in enumerate(dataloader, 0):
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 1, i, len(dataloader), errD.data[0], errG.data[0]))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

[0/1][0/782] Loss_D: 2.0372 Loss_G: 6.2626
[0/1][1/782] Loss_D: 1.2695 Loss_G: 5.8334
[0/1][2/782] Loss_D: 1.0389 Loss_G: 5.7944
[0/1][3/782] Loss_D: 0.7992 Loss_G: 7.4905
[0/1][4/782] Loss_D: 0.7561 Loss_G: 6.6963
[0/1][5/782] Loss_D: 1.0341 Loss_G: 7.6299
[0/1][6/782] Loss_D: 0.7052 Loss_G: 8.5783
[0/1][7/782] Loss_D: 0.5309 Loss_G: 8.5884
[0/1][8/782] Loss_D: 0.5063 Loss_G: 8.3508
[0/1][9/782] Loss_D: 0.7487 Loss_G: 9.8648
[0/1][10/782] Loss_D: 0.5273 Loss_G: 7.5022
[0/1][11/782] Loss_D: 1.2085 Loss_G: 12.8490
[0/1][12/782] Loss_D: 0.4704 Loss_G: 10.5582
[0/1][13/782] Loss_D: 0.2595 Loss_G: 7.1855
[0/1][14/782] Loss_D: 2.1340 Loss_G: 14.9745
[0/1][15/782] Loss_D: 0.4925 Loss_G: 13.2448
[0/1][16/782] Loss_D: 0.3871 Loss_G: 6.3789
[0/1][17/782] Loss_D: 4.0433 Loss_G: 14.8669
[0/1][18/782] Loss_D: 0.3646 Loss_G: 14.3137
[0/1][19/782] Loss_D: 0.3269 Loss_G: 8.0199
[0/1][20/782] Loss_D: 2.8710 Loss_G: 15.5737
[0/1][21/782] Loss_D: 0.4685 Loss_G: 14.5749
[0/1][22/782] Loss_D: 0.2554 Loss_

[0/1][184/782] Loss_D: 0.6554 Loss_G: 2.3739
[0/1][185/782] Loss_D: 0.8959 Loss_G: 6.9352
[0/1][186/782] Loss_D: 0.4659 Loss_G: 5.0212
[0/1][187/782] Loss_D: 0.2869 Loss_G: 4.4527
[0/1][188/782] Loss_D: 0.2830 Loss_G: 4.6026
[0/1][189/782] Loss_D: 0.3689 Loss_G: 4.3419
[0/1][190/782] Loss_D: 0.3553 Loss_G: 4.4878
[0/1][191/782] Loss_D: 0.3724 Loss_G: 4.8126
[0/1][192/782] Loss_D: 0.3907 Loss_G: 4.1079
[0/1][193/782] Loss_D: 0.3878 Loss_G: 5.2398
[0/1][194/782] Loss_D: 0.3341 Loss_G: 4.0044
[0/1][195/782] Loss_D: 0.3751 Loss_G: 5.4219
[0/1][196/782] Loss_D: 0.3974 Loss_G: 4.2566
[0/1][197/782] Loss_D: 0.3658 Loss_G: 4.6751
[0/1][198/782] Loss_D: 0.3132 Loss_G: 4.8593
[0/1][199/782] Loss_D: 0.1535 Loss_G: 5.3713
[0/1][200/782] Loss_D: 0.2532 Loss_G: 4.6959
[0/1][201/782] Loss_D: 0.2909 Loss_G: 4.3841
[0/1][202/782] Loss_D: 0.3778 Loss_G: 5.1628
[0/1][203/782] Loss_D: 0.3482 Loss_G: 4.0834
[0/1][204/782] Loss_D: 0.3913 Loss_G: 7.1261
[0/1][205/782] Loss_D: 0.2935 Loss_G: 4.1379
[0/1][206/